In [9]:
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import itertools
import json
import import_ipynb
import csv
from tqdm import tqdm
from scipy.io import wavfile
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Input, Dense,Conv1D, MaxPooling1D, UpSampling1D, Flatten, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.python.ops import math_ops
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [56]:
####################################
# utility functions
# def cut_old(arr, length):
#   idx = len(arr)%length
#   out = []
#   while(idx+length <= len(arr)):
#     out.append(arr[idx:idx+length])
#     idx += length
#   return np.array(out)

def cut(arr, length):
  ueberhang = len(arr) % length
  a = np.array(arr[ueberhang:]).reshape(-1, length)
  return a

# def loadSong(fName, numTotalSongs = 1):
#   fs, data = wavfile.read(inpathTrain + fName)
#   all_data = [data]

#   if numTotalSongs > 1:
#     seed = sum([ord(char) for char in fName])
#     random.seed(seed)
#     file_nams = random.sample(fileNames[:170], numTotalSongs-1)
#     for name in file_nams:
#       fs, data1 = wavfile.read(inpathTrain + name)
#       all_data.append(data1)

#   concatenated_data = np.concatenate(all_data)
#   if concatenated_data.ndim > 1:
#     mono_data = np.mean(concatenated_data, axis=1, dtype='int16')
#   else:
#     mono_data = concatenated_data

#   return mono_data.astype('int16')

def loadSong(fName, numTotalSongs=1):
  fs, data = wavfile.read(inpathTrain + fName)
  
  if numTotalSongs > 1:
      seed = sum(ord(char) for char in fName)
      random.seed(seed)
      file_nams = random.sample(fileNames[:170], numTotalSongs - 1)
      for name in file_nams:
          fs, data1 = wavfile.read(inpathTrain + name)
          data = np.concatenate((data, data1), axis=0)
  
  if data.ndim > 1:
    mono_data = np.mean(data, axis=1, dtype='int16')
  else:
    mono_data = data

  return mono_data.astype('int16')



def loadSongCut(fName, silence_prob = 0, numTotalSongs = 1, percentage_of_song = 1):
  data = loadSong(fName, numTotalSongs)
  data = cut(data, snippitLength)

  # Replace rows with silence based on silence_prob
  if silence_prob!=0:
    num_rows = data.shape[0]
    num_silence_rows = int(num_rows * silence_prob)
    silence_rows = np.zeros((num_silence_rows, data.shape[1]), dtype='int16')
    data[:num_silence_rows, :] = silence_rows

  scaler[fName] = MinMaxScaler()
  #data = quadratic_scaler(data, 5)
  data = scaler[fName].fit_transform(data)

  Xt, Xv = train_test_split(data, test_size=0.3, random_state=42)
  Xt = np.array(Xt)
  Xv = np.array(Xv)
  if percentage_of_song != 1:
    index_t = int(len(Xt)*percentage_of_song)
    index_v = int(len(Xv)*percentage_of_song)
    Xt = Xt[:index_t]
    Xv = Xv[:index_v]
  return np.array(Xt), np.array(Xv)



def snipLoss(y_true, y_pred):
  snipWeight = tf.convert_to_tensor([int(np.cosh(x)) for x in range(-5, 5, snippitLength)], dtype='float32')
  squared_difference = math_ops.squared_difference(y_true, y_pred)
  loss = math_ops.Mul(x = squared_difference, y = snipWeight)
  loss = math_ops.log1p(loss)
  return loss



def si_snr(original, estimate):
  # original and estimate are tensors of shape (batch_size, time_steps)
  # compute the dot product of original and estimate along the time axis
  dot = tf.reduce_sum(original * estimate, axis=-1, keepdims=True)
  denominator = tf.reduce_sum(original ** 2, axis=-1, keepdims=True)
  # compute the scaled target
  scaled_target = dot * original / denominator
  # compute the noise
  e_noise = estimate - scaled_target
  # compute the SI-SNR in decibels
  si_snr = 10 * tf.math.log(tf.reduce_sum(scaled_target ** 2, axis=-1) / tf.reduce_sum(e_noise ** 2, axis=-1)) / tf.math.log(10.0)
  # return the SI-SNR tensor of shape (batch_size,)
  return si_snr



def si_snr_std(original, estimate):
  dot = np.sum(original * estimate, axis=-1, keepdims=True)
  # compute the energy of target along the time axis
  denominator = np.sum(original ** 2, axis=-1, keepdims=True)
  # compute the scaled target
  scaled_target = dot * original / denominator
  # compute the noise
  e_noise = estimate - scaled_target
  # compute the SI-SNR in decibels
  si_snr = 10 * np.log10(np.sum(scaled_target ** 2, axis=-1) / np.sum(e_noise ** 2, axis=-1))
  # return the SI-SNR array of shape (batch_size,)
  return si_snr


Xt, Xv = loadSongCut('1728_schubert_op114_3.wav', numTotalSongs = 17, percentage_of_song = float(1/17))

In [57]:
Xt

array([[0.46663506, 0.46573283, 0.46715126, ..., 0.56058501, 0.56289272,
        0.56517864],
       [0.59823792, 0.59385882, 0.59115825, ..., 0.47456973, 0.47299641,
        0.47066574],
       [0.59262771, 0.58372903, 0.57340324, ..., 0.591302  , 0.59608342,
        0.60210804],
       ...,
       [0.59112639, 0.5870133 , 0.58416746, ..., 0.60457668, 0.60336918,
        0.60191428],
       [0.59950219, 0.5945315 , 0.59131713, ..., 0.51327468, 0.51274045,
        0.51224521],
       [0.57556003, 0.56410256, 0.55115983, ..., 0.64320445, 0.65178675,
        0.65860653]])

In [11]:
####################################
#####  plot history

def plot_loss(ax, network_history):
    loss = np.concatenate([network_history[key].history['loss'] for key in network_history.keys()])
    val_loss = np.concatenate([network_history[key].history['val_loss'] for key in network_history.keys()])

    ax.set_xlabel('Epochs')
    ax.set_ylabel('Loss')
    ax.set_title('Loss')
    ax.plot(loss, label='Training')
    ax.plot(val_loss, label='Validation')
    ax.legend()

def plot_si_snr(ax, network_history):
    si_snr = np.concatenate([network_history[key].history['si_snr'] for key in network_history.keys()])
    val_si_snr = np.concatenate([network_history[key].history['val_si_snr'] for key in network_history.keys()])

    ax.set_xlabel('Epochs')
    ax.set_ylabel('SI_SNR')
    ax.set_title('SI-SNR')
    ax.plot(si_snr, label='Training')
    ax.plot(val_si_snr, label='Validation')
    ax.legend()

def plot_history(network_history, name):
    fig, ax = plt.subplots(1, 2, figsize=(12, 6), sharex=True, sharey=False)

    plot_loss(ax[0], network_history)
    plot_si_snr(ax[1], network_history)

    plt.tight_layout()
    plt.savefig(name)
    # plt.show()
    plt.clf()


In [12]:
####################################
#####  buildModel (Hyperparameter grid search)

def buildModel(compression_ratio = 0.5, numDense = 1, numConv = 8, numConvLayer = 0, loss_fct = snipLoss, use_bias = False, learning_rate = 0.001):
  
  latentSize = int(compression_ratio*snippitLength)

  # keep tensorflow from allocating more memory as it currently needs
  physical_devices = tf.config.experimental.list_physical_devices('GPU')
  for i in physical_devices:
      tf.config.experimental.set_memory_growth(i, True)
  tf.device('/device:GPU:0')

  input = Input(shape=(snippitLength,1))
  x = input

  # Convolutional part of encoder
  for i in range(numConvLayer):
    x = Conv1D(numConv, 5, activation='relu', padding='same')(x)
    x = MaxPooling1D(2, padding = 'same')(x)

  convShape = x.shape
  # calculate flatten dimension
  flsize = 1
  for i in x.shape:
    if(i != None):
      flsize*= i
  x = Flatten()(x)

  # Dense part of encoder
  denses = [int(i) for i in np.linspace(flsize, latentSize, numDense+1)]
  print(denses)
  print(flsize)
  print(latentSize)
  for i in denses[1:]:
    x = Dense(i, activation='relu', use_bias=use_bias)(x)
    
  encoded = x

  # Dense part of decoder
  x = encoded
  for i in denses[::-1][1:]:
    if(numConvLayer == 0 and i == snippitLength):
      x = Dense(i, activation='sigmoid')(x)
    else:
      x = Dense(i, activation='relu', use_bias=use_bias)(x)

  if(numConvLayer == 0):
    decoded = x

  x = Reshape(convShape[1:])(x)

  # Convolutional part of decoder
  for i in range(numConvLayer):
    x = Conv1D(numConv,5, activation='relu', padding='same')(x)
    x = UpSampling1D(2)(x)
  if(numConvLayer != 0):
    decoded = Conv1D(1,5, activation='sigmoid', padding='same')(x)

  autoencoder = Model(input, decoded)
  autoencoder = Model(input, Flatten()(decoded))

  autoencoder.compile(optimizer=Adam(learning_rate=learning_rate), loss=loss_fct, metrics=[si_snr])
  
  print(f'current model: ratio={compression_ratio},numDense={numDense},numConv={numConv},numConvLayer={numConvLayer}')
  autoencoder.summary()
  return autoencoder

get_custom_objects()['snipLoss'] = snipLoss
get_custom_objects()['si_snr'] = si_snr

# testing:
# Xt, Xv = loadSongCut('1727_schubert_op114_2.wav')
# buildModel(numDense=1).fit(Xt[:2], Xt[:2],
#             epochs=1,
#             batch_size=512,
#             shuffle=True,

#             validation_data=(Xv[:2], Xv[:2]))

In [13]:
####################################
#####  evaluate Songs

def evaluateTestSongs(autoencoder, num = 0):
  test_evaluated = []
  if num!=0:
    numTestSongs = num
  print(f'evaluating {numTestSongs} test songs')
  for songname in tqdm(reversed(fileNames[-numTestSongs:])):
      orig = loadSong(songname)
      origSnip = cut(orig, snippitLength)
      orig = np.concatenate(origSnip)
      
      if(songname in scaler.keys()):
        scaler_Example = scaler[songname]
        origSnip_transformed = scaler_Example.transform(origSnip)
      else:
        scaler_Example = MinMaxScaler()
        origSnip_transformed = scaler_Example.fit_transform(origSnip)

      # autoencode song
      a = autoencoder.predict(origSnip_transformed)
      a = a.reshape(-1, snippitLength)
      XpredSnip = scaler_Example.inverse_transform(a)
      estimate_uncorr = np.concatenate(XpredSnip).astype('int16')


      silence = np.zeros((1,snippitLength), dtype = 'int16')
      a = scaler_Example.transform(silence)
      a = autoencoder.predict(a)
      a = a.reshape(-1, snippitLength)
      Xsilence = scaler_Example.inverse_transform(a)[0]

      # remove noise generated by silence
      XpredSnip_minussilence = np.array(XpredSnip) - Xsilence
      Xpred = np.concatenate(XpredSnip_minussilence).astype('int16')

      test_loss, test_si_snr_uncorr = autoencoder.evaluate(origSnip_transformed, origSnip_transformed, verbose=2)
      
      test_si_snr_corrected = si_snr_std(orig, Xpred)
      output_wav_name = f'{Test_Song}_{name}_{compression_ratio:.1f}_SNR={test_si_snr_corrected:.1f}.wav'
      wavfile.write(f'{output_folder}{Test_Song}.wav', samplerate, orig)
      wavfile.write(output_folder + output_wav_name, samplerate, Xpred)
      print(f"Test song predicted and saved: {output_wav_name}")

      test_evaluated.append([songname, test_loss, test_si_snr_uncorr, test_si_snr_corrected])
  return test_evaluated

In [14]:
####################################
#####  waveform plots
#####  predict test song and save it

def plotWave(autoencoder, name, compression_ratio, Test_Song = None):
  if Test_Song == None:
    Test_Song = fileNames[-1]
  # exampleSong = name
  # exampleSong = '1727_schubert_op114_2.wav'
  orig = loadSong(Test_Song)
  origSnip = cut(orig, snippitLength)
  orig = np.concatenate(origSnip)

  if(Test_Song in scaler.keys()):
    scaler_Example = scaler[Test_Song]
    origSnip_transformed = scaler_Example.transform(origSnip)
  else:
    scaler_Example = MinMaxScaler()
    origSnip_transformed = scaler_Example.fit_transform(origSnip)

  # autoencode song
  a = autoencoder.predict(origSnip_transformed)
  a = a.reshape(-1, snippitLength)
  XpredSnip = scaler_Example.inverse_transform(a)
  estimate_uncorr = np.concatenate(XpredSnip).astype('int16')

  silence = np.zeros((1, snippitLength), dtype = 'int16')
  a = scaler_Example.transform(silence)
  a = autoencoder.predict(a)
  a = a.reshape(-1, snippitLength)
  Xsilence = scaler_Example.inverse_transform(a)[0]

  # remove noise generated by silence
  # XpredSnip_minussilence = [i-Xsilence for i in XpredSnip]
  XpredSnip_minussilence = np.array(XpredSnip) - Xsilence
  Xpred = np.concatenate(XpredSnip_minussilence).astype('int16')
  estimate_corr = Xpred

  # test_loss, test_si_snr = autoencoder.evaluate(origSnip_transformed, origSnip_transformed)
  si_snr_uncorr = si_snr_std(orig, estimate_uncorr)
  print(f'ucorrected SI-SNR = {si_snr_uncorr} dB')

  si_snr_corr = si_snr_std(orig, estimate_corr)
  print(f'corrected SI-SNR = {si_snr_corr} dB')


  # output_wav_name = f'snln={snippitLength}_cmpr={compression_ratio:.1f}_loss={loss_fct.__name__}_SNR={testwav_si_snr:.1f}.wav'
  output_wav_name = f'{Test_Song}_{name}_{compression_ratio:.1f}_SNR={si_snr_corr:.1f}.wav'
  wavfile.write(f'{output_folder}{Test_Song}.wav', samplerate, orig)
  wavfile.write(f'{output_folder}UNCORR_{output_wav_name}', samplerate, Xpred)
  wavfile.write(output_folder + output_wav_name, samplerate, Xpred)
  print(f"Test song predicted and saved: {output_wav_name}")


  ###### plots
  plt.plot(orig, linewidth = 0.1)
  plt.plot(orig-Xpred, linewidth = 0.1)
  plt.savefig(name + "whole.pdf")
  plt.clf()
  ####################################
  #####  see difference in waveform detailed
  nrows = 2
  ncols = 6
  snips = [0, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000]

  fig, ax = plt.subplots(nrows, ncols, figsize=(6*ncols, 6*nrows), sharey = True, sharex = True)
  s = 0
  for i in range(nrows):
    for j in range(ncols):
      ax[i][j].plot(origSnip[snips[s]], linewidth = 0.5, c = 'b')
      ax[i][j].plot(XpredSnip_minussilence[snips[s]], linewidth = 0.5, c = 'r')
      s +=1
  plt.savefig(name + "snip_corrected.pdf")
  plt.clf()

  fig, ax = plt.subplots(nrows, ncols, figsize=(6*ncols, 6*nrows), sharey = True, sharex = True)
  s = 0
  for i in range(nrows):
    for j in range(ncols):
      ax[i][j].plot(origSnip[snips[s]], linewidth = 0.5, c = 'b')
      ax[i][j].plot(XpredSnip[snips[s]], linewidth = 0.5, c = 'r')
      # ax[i][j].plot(XpredSnip_minussilence[snips[s]], linewidth = 0.5, c = 'r')
      s +=1
  plt.savefig(name + "snip_notcorrected.pdf")
  plt.clf()

# model_save_path = output_folder + 'train_compression_rates/' + f'model_train_1_96_3.keras' #100
# autoencoder = tf.keras.models.load_model(model_save_path)
# plotWave(autoencoder, f'{output_folder}train_compression_rates/model_0.2_1_96_3wave_', 0.2)

In [15]:
def read_histories_from_csv(file_path):
    histories = {}
    with open(file_path, 'r') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)  # Skip the header row
        for row in reader:
            filename = row[0]
            loss = float(row[1])
            val_loss = float(row[2])
            histories[filename] = {'loss': [loss], 'val_loss': [val_loss]}
    return histories

In [16]:
# paths
# drive.mount('/content/drive')
# inpathTrain = "/content/drive/MyDrive/Machine Learning/Autoencoder/train_data/"
# inpathOut = "/content/drive/MyDrive/Machine Learning/Autoencoder/output/"
inpathTrain = "songs/wav/"
output_folder = "output/Versuch_new/"
fileNames = os.listdir(inpathTrain)
random.seed(42)
fileNames = random.sample(fileNames, len(fileNames))

hyperparamsearch_folder = output_folder + 'hyperparamsearch'
train_compression_rates_folder = output_folder + 'train_compression_rates'

if not os.path.exists(hyperparamsearch_folder):
    os.mkdir(hyperparamsearch_folder)

if not os.path.exists(train_compression_rates_folder):
    os.mkdir(train_compression_rates_folder)

scaler = {}

# global variables
samplerate = 44_100
snippitLength = 64

loss_fct = snipLoss

In [17]:
param_space = {'compression_ratio' : [0.1, 0.2, 0.3, 0.4],
               'numDense' : [3, 4, 5, 6],
               'numConv' : [8, 16],
               'numConvLayer' : [0, 1, 2]}



# param_space = {'compression_ratio' : np.linspace(0.1,0.9,2),
#                'numDense' : [2, 3],
#                'numConv' : [8, 6],
#                'numConvLayer' : np.linspace(0.1,0.2,2)} # small 1
# param_space = {'compression_ratio' : [0.1, 0.2],
#                'numDense' : [3,4],
#                'numConv' : [4, 8],
#                'numConvLayer' : [0, 1]} # small 2


param_space = {'compression_ratio' : [0.2],
               'numDense' : [2, 3, 4, 5, 6],
               'numConv' : [8, 16, 24, 32],
               'numConvLayer' : [0, 1, 2, 3, 4]}

param_space = {'compression_ratio' : [0.2],
               'numDense' : [2, 3, 4],
               'numConv' : [32, 64],  # 128 too much memory need
               'numConvLayer' : [1, 2]}

param_space = {'compression_ratio' : [0.2],
               'numDense' : [1],
               'numConv' : [32, 64],  # 128 too much memory need
               'numConvLayer' : [1, 2]}


param_space = {'compression_ratio' : [0.2],
               'numDense' : [1],
               'numConv' : [64, 96, 128],
               'numConvLayer' : [4, 6]}


# param_space = {'compression_ratio' : [0.2],
#                'numDense' : [5, 6],
#                'numConv' : [24, 32],
#                'numConvLayer' : [ 3, 4]}

#### test #####
param_space = {'compression_ratio' : [0.2],
               'numDense' : [1],
               'numConv' : [64, 96, 128],
               'numConvLayer' : [1, 2, 3]}



value_combis = itertools.product(*[v for v in param_space.values()])
param_combis = []
for combi in value_combis:
  param_combi = {key: value for key, value in zip(param_space.keys(), combi)}
  if param_combi['numConvLayer'] == 0:
    param_combi['numConv'] = 0
  param_combis.append(param_combi)

# param_combis = [
#  {'compression_ratio': 0.2, 'numDense': 1, 'numConv': 32, 'numConvLayer': 2}, #47.8
#  {'compression_ratio': 0.2, 'numDense': 1, 'numConv': 32, 'numConvLayer': 3}, #47.8
#  {'compression_ratio': 0.2, 'numDense': 1, 'numConv': 64, 'numConvLayer': 2}, #47.8
#  {'compression_ratio': 0.2, 'numDense': 1, 'numConv': 64, 'numConvLayer': 3}, #47.8
#  {'compression_ratio': 0.2, 'numDense': 1, 'numConv': 96, 'numConvLayer': 2}, # 48.2
#  {'compression_ratio': 0.2, 'numDense': 1, 'numConv': 96, 'numConvLayer': 3}, # 48.2
#  {'compression_ratio': 0.2, 'numDense': 1, 'numConv': 128, 'numConvLayer': 2}, # 48.5
#  {'compression_ratio': 0.2, 'numDense': 1, 'numConv': 128, 'numConvLayer': 3}, # 48.5
#  {'compression_ratio': 0.2, 'numDense': 1, 'numConv': 224, 'numConvLayer': 2}, # 49.6
#  {'compression_ratio': 0.2, 'numDense': 1, 'numConv': 224, 'numConvLayer': 3}] # 49.6

#  {'compression_ratio': 0.2, 'numDense': 1, 'numConv': 96, 'numConvLayer': 2}, # 48.2
#  {'compression_ratio': 0.2, 'numDense': 1, 'numConv': 96, 'numConvLayer': 2}, # 48.2
#  {'compression_ratio': 0.2, 'numDense': 1, 'numConv': 96, 'numConvLayer': 2}, # 48.2
#  {'compression_ratio': 0.2, 'numDense': 1, 'numConv': 96, 'numConvLayer': 3},
#  {'compression_ratio': 0.2, 'numDense': 1, 'numConv': 96, 'numConvLayer': 3},
#  {'compression_ratio': 0.2, 'numDense': 1, 'numConv': 96, 'numConvLayer': 3},
# param_combis = [
#  {'compression_ratio': 0.2, 'numDense': 1, 'numConv': 96, 'numConvLayer': 4},
#  {'compression_ratio': 0.2, 'numDense': 1, 'numConv': 96, 'numConvLayer': 5},
#  {'compression_ratio': 0.2, 'numDense': 1, 'numConv': 96, 'numConvLayer': 6}]

batch_size = 4096

numHyperEpochs = 1

numTotalSongs = 17
percentage_of_song = float(1/(numTotalSongs))
numHyperTrainSongs = 120
numHyperTrainSongs = 170
# numHyperTrainSongs = 170
# numHyperEpochs = 10

# param_combis = param_combis[5:]
time_per_combi = 2.6
print(f'estimated time = {time_per_combi*len(param_combis)/60:.1f} h ({len(param_combis)} sets)')

param_combis

estimated time = 0.4 h (9 sets)


[{'compression_ratio': 0.2, 'numDense': 1, 'numConv': 64, 'numConvLayer': 1},
 {'compression_ratio': 0.2, 'numDense': 1, 'numConv': 64, 'numConvLayer': 2},
 {'compression_ratio': 0.2, 'numDense': 1, 'numConv': 64, 'numConvLayer': 3},
 {'compression_ratio': 0.2, 'numDense': 1, 'numConv': 96, 'numConvLayer': 1},
 {'compression_ratio': 0.2, 'numDense': 1, 'numConv': 96, 'numConvLayer': 2},
 {'compression_ratio': 0.2, 'numDense': 1, 'numConv': 96, 'numConvLayer': 3},
 {'compression_ratio': 0.2, 'numDense': 1, 'numConv': 128, 'numConvLayer': 1},
 {'compression_ratio': 0.2, 'numDense': 1, 'numConv': 128, 'numConvLayer': 2},
 {'compression_ratio': 0.2, 'numDense': 1, 'numConv': 128, 'numConvLayer': 3}]

In [45]:
####################################
#####  Hyperparameter grid search
import stopwatch as sw

t = sw.stopwatch(title='gridsearch', time_unit='s')

# Load existing results from the JSON file if it exists
existing_results = []
existing_file_path = output_folder + 'hyperparamsearch/' + 'searchResults_170songs_3epochs.json'
existing_file_path = output_folder + 'hyperparamsearch/' + 'searchResults_170songs_1epochs_final.json'
existing_file_path = output_folder + 'hyperparamsearch/' + 'searchResults_new.json'
if os.path.exists(existing_file_path):
    with open(existing_file_path, 'r') as file:
        existing_results = json.load(file)

search_results = []
model_save_path = output_folder + 'hyperparamsearch/' + f'model.keras'
if os.path.exists(model_save_path):
    os.remove(model_save_path)

for hyperParamSet in tqdm(param_combis):
  autoencoder = buildModel(hyperParamSet['compression_ratio'],
                           hyperParamSet['numDense'],
                           hyperParamSet['numConv'],
                           hyperParamSet['numConvLayer'])

  histories = {}

  t.task('hyperparam')
  for idx, filename_train in tqdm(enumerate(fileNames[:numHyperTrainSongs])):
    Xt, Xv = loadSongCut(filename_train, numTotalSongs = numTotalSongs, percentage_of_song = percentage_of_song)
    histories[filename_train] = autoencoder.fit(Xt, Xt,
                epochs=numHyperEpochs,
                batch_size=batch_size,
                shuffle=True,
                validation_data=(Xv, Xv))
    del Xt
    del Xv
    if (idx % int(5/numHyperEpochs) == 0) and (idx != 0):
      autoencoder.save(model_save_path)
      del autoencoder
      autoencoder = tf.keras.models.load_model(model_save_path)
  t.stop()
  del autoencoder
  tf.keras.backend.clear_session()

  pdfname = f'HyperParOpt, compression_ratio= {hyperParamSet["compression_ratio"]:.1f}, numDense= {hyperParamSet["numDense"]}, numConvLayer= {hyperParamSet["numConvLayer"]}, numConv= {hyperParamSet["numConv"]}.pdf'
  plot_history(histories, output_folder + 'hyperparamsearch/' + pdfname)

  loss = []
  val_loss = []
  train_si_snr = []
  val_si_snr = []
  for key in histories.keys():
    loss.append(histories[key].history['loss'])
    val_loss.append(histories[key].history['val_loss'])
    train_si_snr.append(histories[key].history['si_snr'])
    val_si_snr.append(histories[key].history['val_si_snr'])
  loss         = np.concatenate(loss)
  val_loss     = np.concatenate(val_loss)
  train_si_snr = np.concatenate(train_si_snr)
  val_si_snr   = np.concatenate(val_si_snr)

  best_val_epoch    = np.argmax(val_si_snr)
  best_val_si_snr   = np.max(val_si_snr)
  best_val_loss     = np.min(val_loss)
  best_train_si_snr = np.max(train_si_snr)
  best_train_loss   = np.min(loss)

  search_results.append({
    **hyperParamSet,
    'best_val_epoch': best_val_epoch,
    'best_val_si_snr': best_val_si_snr,
    'best_val_loss': best_val_loss,
    'best_train_si_snr': best_train_si_snr,
    'best_train_loss': best_train_loss
  })


  latest_results = [{k: int(v) if isinstance(v, np.int64) else v for k, v in d.items()} for d in search_results]

  # Merge existing results and latest results
  all_results = existing_results + latest_results

  # Write all results to the JSON file
  with open(existing_file_path, 'w') as file:
      json.dump(all_results, file, indent='')


  0%|          | 0/9 [00:00<?, ?it/s]

[2048, 12]
2048
12
current model: ratio=0.2,numDense=1,numConv=64,numConvLayer=1
Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 64, 1)]           0         
                                                                 
 conv1d_3 (Conv1D)           (None, 64, 64)            384       
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 32, 64)           0         
 1D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 2048)              0         
                                                                 
 dense_2 (Dense)             (None, 12)                24576     
                                                                 
 dense_3 (Dense)             (None, 2048)   

2023-07-30 00:10:44.362762: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8902
2023-07-30 00:10:47.370619: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x3c1e4350 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-07-30 00:10:47.370869: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 2070, Compute Capability 7.5
2023-07-30 00:10:47.449790: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-07-30 00:10:47.691389: W tensorflow/compiler/xla/service/gpu/llvm_gpu_backend/gpu_backend_lib.cc:530] Can't find libdevice directory ${CUDA_DIR}/nvvm/libdevice. This may result in compilation or runtime failures, if the program we try to run uses routines from libdevice.
Searched for CUDA in the following directories:
  ./cuda_sdk_lib
  /usr/loca

45/45 [==============================] - ETA: 0s - loss: 0.2696 - si_snr: 22.3271

2023-07-30 00:10:50.420295: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 7.83GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-07-30 00:10:50.536179: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 7.83GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-07-30 00:10:50.538364: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 7.83GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


...

2023-07-30 00:10:56.881081: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 6.51GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-07-30 00:10:56.881148: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 6.51GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


48/48 [==============================] - 2s 36ms/step - loss: 0.2291 - si_snr: 22.2728 - val_loss: 0.2105 - val_si_snr: 22.9724


13/52 [======>.......................] - ETA: 1s - loss: 0.2072 - si_snr: 22.3526

2it [00:27, 13.94s/it]
  0%|          | 0/9 [00:28<?, ?it/s]


KeyboardInterrupt: 

In [46]:
Xt

array([[0.43202943, 0.43355804, 0.42930936, ..., 0.62785079, 0.60562264,
        0.58797545],
       [0.61044971, 0.60406013, 0.59663287, ..., 0.52762016, 0.49758491,
        0.46944998],
       [0.56100937, 0.54187975, 0.51704611, ..., 0.61657576, 0.61958491,
        0.62178961],
       ...,
       [0.60083851, 0.58631644, 0.56797398, ..., 0.48204415, 0.48049057,
        0.47543098],
       [0.44429063, 0.43793584, 0.4264779 , ..., 0.53428268, 0.53460377,
        0.53434189],
       [0.67353558, 0.66042926, 0.64331356, ..., 0.61247575, 0.61090566,
        0.60705211]])

In [ ]:
# hyperparameter plot
from plot_hyperparameter import *
hyperparameter_Plot(results, output_folder)

In [ ]:
# SI-SNR corrected vs unorrected 
estimate_uncorr = np.concatenate(XpredSnip).astype('int16')
estimate_corr = Xpred
original = orig

uncorr = si_snr_std(original, estimate_uncorr)
print(f'ucorrected SI-SNR {uncorr} dB')

corr = si_snr_std(original, estimate_corr)
print(f'corrected SI-SNR {corr} dB')

# original_d2 = np.array(original) - 0.1
# corr = si_snr_std(original, original_d2)
# print(f'corrected SI-SNR {corr} dB')

# original_d2 = np.array(original) + 1
# corr = si_snr_std(original, original_d2)
# print(f'corrected SI-SNR {corr} dB')

# original_d = [i+0.01 for i in original]
# corr = si_snr_std(original, np.concatenate(original_d).astype('int16'))
# print(f'corrected SI-SNR {corr} dB')

# output_wav_name = f'snln={snippitLength}_cmpr={compression_ratio:.1f}_loss={loss_fct.__name__}_SNR={testwav_si_snr:.1f}.wav'
# wavfile.write(output_folder + 'original.wav', samplerate, orig)
# wavfile.write(output_folder + output_wav_name, samplerate, Xpred)


In [ ]:
# train best model

# parSet_sum = {}
# for item in search_results:
#     numDense = item['numDense']
#     numConv = item['numConv']
#     numConvLayer = item['numConvLayer']
#     best_val_si_snr = item['best_val_si_snr']
# 
#     key = (numDense, numConv, numConvLayer)
#     if key in parSet_sum:
#        parSet_sum[key] += best_val_si_snr
#     else:
#         parSet_sum[key] = best_val_si_snr
# 
# keys = [k for k in parSet_sum.keys()]
# si_snr_sum = [parSet_sum[k] for k in keys]
# bestParSet = keys[np.argmax(si_snr_sum)]
# print(f'best set : {bestParSet}')


# search_results_json = output_folder + 'hyperparamsearch/' + 'searchResults.json'
# search_results_json = 'output/Versuch1_11.07.2023/searchResults.json'
# search_results_json = 'output/Versuch3_13.07.2023/hyperparamsearch/searchResults_170songs_5epochs_final.json'
# with open(search_results_json, 'r') as file:
#     search_results = json.load(file)

# compression_rates = np.linspace(0.1,0.9,9)
compression_rates = [0.2]
# silence_prob = 0.01


numTotalSongs = 17
percentage_of_song = float(1/(numTotalSongs))
total_num_songs = len(fileNames)
numTopoTrainSongs = int(((total_num_songs*0.7)+1))  # 170
numTopoTrainSongs = 50
numTopoEpochs = 1

numTestSongs = int(total_num_songs*0.3)

batch_size = 61
learning_rate = 0.00008 #no


model_save_path = output_folder + 'train_compression_rates/' + f'model_train_1_96_3_zweite.keras' #100
histories_save_path = output_folder + 'train_compression_rates/' + 'histories_train_1_96_3.csv'
def lr_schedule(epoch):
  return learning_rate
# autoencoder = tf.keras.models.load_model(model_save_path)

lr_scheduler = LearningRateScheduler(lr_schedule)
for c in compression_rates:
  # autoencoder = buildModel(c, bestParSet[0],bestParSet[1],bestParSet[2], learning_rate = learning_rate)
  autoencoder = buildModel(c, 1, 96, 3, learning_rate = learning_rate)
  # autoencoder = tf.keras.models.load_model(model_save_path)

  histories = {}
 
  for idx, filename_train in tqdm(enumerate(fileNames[0:numTopoTrainSongs:])):
    Xt, Xv = loadSongCut(filename_train, numTotalSongs = numTotalSongs, percentage_of_song = percentage_of_song)
    histories[filename_train] = autoencoder.fit(Xt, Xt,
                epochs=numTopoEpochs,
                batch_size=batch_size,
                shuffle=True,
                validation_data=(Xv, Xv),
                callbacks=[lr_scheduler])
    del Xt
    del Xv
    save_period = int(9/numHyperEpochs)
    save_period = 1
    if (idx % save_period == 0) and (idx != 0):
      autoencoder.save(model_save_path)
      del autoencoder
      autoencoder = tf.keras.models.load_model(model_save_path)

      # with open(histories_save_path, 'w', newline='') as csvfile:
      #     writer = csv.writer(csvfile)
      #     writer.writerow(['filename', 'loss', 'val_loss'])
      #     for key, value in histories.items():
      #         writer.writerow([key, value.history['loss'][0], value.history['val_loss'][0]], value.history['loss'][0])
  tf.keras.backend.clear_session()
  autoencoder.save(f'{output_folder}train_compression_rates/modelCompressionRate:{c:.1f}.keras')
  
  pdfname = f'BestSet, compression_ratio ={c:.1f}.pdf'
  # histories = read_histories_from_csv(histories_save_path)
  plot_history(histories, f'{output_folder}train_compression_rates/{pdfname}')
  plotWave(autoencoder, f'{output_folder}train_compression_rates/modelCompressionRate:{c:.1f}wave_', c)

  testPerformance = evaluateTestSongs(autoencoder, num = 10)

  with open(f'{output_folder}train_compression_rates/modelCompressionRate:{c:.1f}Performance.csv', 'w', newline='') as file:
    writer = csv.writer(file)

    for row in testPerformance:
        writer.writerow(row)
